In [1]:
import pickle
with open('../data/output/perturbed_graphs.txt', 'rb') as f:
    perturbed_graphs = pickle.load(f)
    
len(perturbed_graphs)

1

In [38]:
import torch
from tqdm import tqdm
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

embeddings = torch.rand(5, 32, device=device) (M, n_graph_embedding, )
prediction = torch.rand(32, device=device)

distances = torch.norm(embeddings - prediction, dim=1)
probabilities = 1 / distances
distances
# 
# (N, n_graph_embedding) x

tensor([2.2650, 2.3784, 2.3975, 2.1637, 2.3885], device='mps:0')

In [35]:
distances.mean().item()

2.4682819843292236